# Scoring

In [ ]:
import pandas as pd
import respiration.utils as utils

analysis_dir = utils.dir_path('outputs', 'analysis')

frequency_file = utils.file_path(analysis_dir, 'frequency_evaluation.csv')
frequency_df = pd.read_csv(frequency_file)

distances_file = utils.file_path(analysis_dir, 'distance_evaluation.csv')
distances_df = pd.read_csv(distances_file)

In [ ]:
# Bigger correlation is better --> 1 - correlation
for idx, row in distances_df.iterrows():
    if row['metric'] != 'pearson':
        continue

    distances_df.at[idx, 'mean'] = 1 - row['mean']

In [ ]:
import numpy as np

# Use the absolute value of the correlation
frequency_df['corr'] = frequency_df['corr'].apply(np.abs)

In [ ]:
frequency_df['mae_rank'] = frequency_df.groupby('metric')['mae'].rank(ascending=True)
frequency_df['rmse_rank'] = frequency_df.groupby('metric')['rmse'].rank(ascending=True)
frequency_df['corr_rank'] = frequency_df.groupby('metric')['corr'].rank(ascending=False)

distances_df['rank'] = distances_df.groupby('metric')['mean'].rank(ascending=True)

In [ ]:
distances_df

In [ ]:
metric_ranks = []

for _, row in frequency_df.iterrows():
    model = row['model']
    metric = row['metric']

    metric_ranks.extend([
        {
            'model': model,
            'metric': f'{metric}-mae',
            'rank': row['mae_rank']
        },
        {
            'model': model,
            'metric': f'{metric}-rmse',
            'rank': row['rmse_rank']
        },
        {
            'model': model,
            'metric': f'{metric}-corr',
            'rank': row['corr_rank']
        },
    ])

for _, row in distances_df.iterrows():
    model = row['model']
    metric = row['metric']
    
    if metric == 'mse':
        continue

    metric_ranks.append({
        'model': model,
        'metric': metric,
        'rank': row['rank']
    })

ranks_df = pd.DataFrame(metric_ranks)
ranks_df

In [ ]:
# Create a dataframe with the ranks of the models where each rank is a column and the model is the index
ranks_df_xxx = ranks_df.pivot(index='model', columns='metric', values='rank')

# Store the ranks in a csv file
ranks_df_xxx.to_csv(utils.file_path(analysis_dir, 'model_ranks.csv'))

In [ ]:
optical_flow_models = [
    'FlowNet2',
    'FlowNet2C',
    'FlowNet2CS',
    'FlowNet2CSS',
    'FlowNet2S',
    'FlowNet2SD',
    'lucas_kanade',
    'pixel_intensity',
    'raft_large',
    'raft_small',
]

other_models = set(ranks_df['model'].unique()) - set(optical_flow_models)
other_models

In [ ]:
# Get the mean and standard deviation of the ranks for each model
mean_ranks = ranks_df.groupby('model')['rank'].mean()
std_ranks = ranks_df.groupby('model')['rank'].std()

# Rename the columns
mean_ranks.name = 'score'
std_ranks.name = 'std'

# Merge the mean and standard deviation of the ranks
scores_df = pd.concat([mean_ranks, std_ranks], axis=1)

# Reset the index
scores_df.reset_index(inplace=True)
scores_df

In [ ]:
# Show only the other models
scores_df[scores_df['model'].isin(other_models)]

In [ ]:
# Sort the standard deviation and plot the results
scores_df.sort_values('score', inplace=True)
scores_df.plot.bar(x='model', y='score', yerr='std', figsize=(20, 5))